In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import sys, os, pathlib, shutil, platform
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX

from prophet import Prophet

from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

from prophet.plot import plot_plotly, plot_components_plotly
from prophet.plot import plot_cross_validation_metric

# import plotly.graph_objs as go
import plotly.express as px

from dateutil.relativedelta import relativedelta
import humanize
from datetime import date, timedelta


In [ ]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2
 
plt.rcParams['figure.figsize']=(20,10)

In [ ]:
## rewrote prophet's make future dataframe cuz it doesnt handle very well only monthly data; 
# fc_model.make_future_dataframe(periods=10, freq='m') ##compare my_make_future_dataframe with the one provided by Prophet
def my_make_future_dataframe(df, periods):
    last_date=df['ds'].max()
    complete_df = df.append(pd.DataFrame([last_date + relativedelta(months = i + 1) for i in range(periods)],
                                              columns =['ds']), ignore_index=True, sort=True)
    return complete_df

def forecast_future(future_samples_count, df, growth = 'linear'):
    model = Prophet(growth=growth)
    model.fit(df)
    
    future = my_make_future_dataframe(df, periods=future_samples_count)

    forecast = model.predict(future)
    return forecast, model

def forecast_in_sample(hold_out_samples_count, df, growth = 'linear'):
    train_data = df.drop(df.index[-hold_out_samples_count:])
    print(train_data.head(4), train_data.tail(4))
    print(train_data.shape)
    
    model = Prophet(growth=growth)
    model.fit(train_data)
    
    future = df[['ds']].reset_index()                         # predicts for all ds values
    forecast = model.predict(future)
    return forecast, model

def forecasted_percentiles(fc_model, input_df, percentiles):     
    forecasted_samples = fc_model.predictive_samples(input_df)
    forecasted_stats=pd.DataFrame(data=np.transpose(np.percentile(forecasted_samples['yhat'], percentiles, axis=1 )) #made a change, it said 'yhat' before 'Predicted'
             ,  columns = ['pct_'+str(x) for x in percentiles])
    forecasted_stats.insert(loc=0, column='Predicted', value=input_df['yhat'])
    forecasted_stats.insert(loc=0, column='ds', value=input_df['ds'])
    return forecasted_stats 

In [ ]:
def load_datasets(data_path, years_list, analysis_type_list):
    """
    Concatanates a list of datasets into one dataframe, and also does yearly agg

    Returns:
    Concatanated list as a dataframe
    """
    #concat
    full_path_list = [data_path+str(crt_year) + '_analysis'+ str(analysis_type) +'_' + str(regional_flag)+'_'+str(aircraft_flag) + '.csv' 
                      for crt_year in years_list 
                      for analysis_type in analysis_type_list]
    print(full_path_list[0])
    all_datasets= pd.concat([pd.read_csv(str(crt_file_name)) for crt_file_name in full_path_list], keys=years_list).reset_index()
    
    #yearly agg
    all_datasets.drop(columns=['level_1'],axis=1,inplace=True)
    all_datasets = all_datasets.rename(columns = {'level_0':'year'})
    all_datasets = all_datasets.groupby(['year', 'origin', 'dest'], as_index=True, group_keys=True)['y', 'num flights' ].agg(['sum','count'])
    all_datasets.reset_index(inplace=True)
    all_datasets.columns= ['_'.join(col) for col in all_datasets.columns.values]
    all_datasets.drop(['origin_', 'dest_', 'y_count', 'num flights_count','num flights_sum'], axis=1, inplace = True)
    all_datasets.rename(columns={'y_sum': 'y', 'year_':'ds' }, inplace = True)

# my_testing["year"] = my_testing['ds'].dt.year #tagged this cuz level_0 column takes care of it

    return all_datasets    

In [ ]:
# years_list = [yr for yr in range (1990,2024)]
# these are the years lists we use when regional_flag = regional and aircraft_flag = nonjets
# years_list_s1 = [yr for yr in range (2004,2024)]
# years_list_s5 = [yr for yr in range (2002,2024)]
# years_list_s2 = [yr for yr in range (2002,2020)]
# years_list_s3 = [yr for yr in range (2002,2020)]
# years_list_s4 = [yr for yr in range (2002,2020)]

# these are the years lists we use when regional_flag = regional and aircraft_flag = jetsandnonjets
# years_list_s1 = [yr for yr in range (2004,2024)]
# years_list_s5 = [yr for yr in range (2002,2024)]
# years_list_s2 = [yr for yr in range (2002,2024)]
# years_list_s3 = [yr for yr in range (2002,2024)]
# years_list_s4 = [yr for yr in range (2002,2020)]

# these are the years lists we use when regional_flag = regional and aircraft_flag = jets
years_list_s1 = [yr for yr in range (2004,2024)]
years_list_s2 = [yr for yr in range (2002,2024)]
years_list_s3 = [yr for yr in range (2002,2024)]



data_path='./../../../data/paav_cargo/agg_data/freight_aggregation__' 

analysis_type_list = [analysis_type for analysis_type in range(11)]
# regional_flag = 'alldistance' # , 'regional']
# aircraft_flag = 'jetsandnonjets' # 'nonjets']
regional_flag = 'regional' # , 'alldistance']
# aircraft_flag = 'nonjets' # 'jetsandnonjets']
aircraft_flag = 'jets'

# analysis_type_list = [0]
# preprocessed_data_s1 = load_datasets(data_path, years_list_s1, analysis_type_list)

# analysis_type_list = [7]
# preprocessed_data_s2 = load_datasets(data_path, years_list_s2, analysis_type_list)

analysis_type_list = [8]
preprocessed_data_s3 = load_datasets(data_path, years_list_s3, analysis_type_list)

# analysis_type_list = [9]
# preprocessed_data_s4 = load_datasets(data_path, years_list_s4, analysis_type_list)

# analysis_type_list = [10]
# preprocessed_data_s5 = load_datasets(data_path, years_list_s5, analysis_type_list)

In [ ]:
preprocessed_data_s3.head(2)

In [ ]:
preprocessed_data_s3.tail(2)

# Cross validation

In [ ]:
# m_s1 = Prophet()
# m_s1.fit(preprocessed_data_s1[preprocessed_data_s1['ds'] < 2023]) 
#less than 2023 bc 2023 doesnt have all months in and gives us huge errors which are not rep of prophets performance
# fit fxn creates coeffs for model but those are not passed to cross_validation fxn below. when u fit a df to model, the model will remember/ record the ds and y columns and 
# that's the purpose here. the ds and y memory is passed to cross val fxn. idk why they ddint just ask for ds and y column from a df as oppsoed to fitting like we do here
# which is kind of useless.

m_s3 = Prophet()
m_s3.fit(preprocessed_data_s3[preprocessed_data_s3['ds'] < 2023]) 

In [ ]:
initial_time_s1 = f'{365*8} days' # yr 2012; 8 years is the largest initial training period i can make bc i have less data now (2004+ as opposed to 1990+)
initial_time = f'{365*10} days'  
initial_time_s4 =f'{365*7} days'
#yr 2009 for s5 (because its data only begins in october 2002), s4 (because its data only begins in october 2002), s2 (because its data only begins in october 2002)
#yr 2008 for s3 (bc its data only begins march 2001 but bc theres zero freight in that one flight, it actually begins in october 2002)
initial_time
# cross_validation_results_s1 = cross_validation(m_s1, initial=initial_time_s1, period='365 days', horizon = '3650 days', parallel="processes")
cross_validation_results_s3= cross_validation(m_s3, initial=initial_time, period=f'{365*1} days', horizon = '3650 days', parallel="processes")
# cross_validation_results_s4= cross_validation(m_s4, initial=initial_time_s4, period=f'{365*1} days', horizon = '3650 days', parallel="processes")

cross_validation_results_s3

In [ ]:
cross_validation_results_s3.rename(columns={'cutoff': 'real_cutoff'},inplace=True) 
cross_validation_results_s3['cutoff'] = np.NaN                                     
for index, row in cross_validation_results_s3.iterrows():
    cross_validation_results_s3.at[index, 'cutoff'] = row['ds'] - timedelta(days=365*(((index)%10)+1))
                                      
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(cross_validation_results)
cross_validation_results_s3.head(3)

In [ ]:
# cross_validation_results["sanity check diff1"] = cross_validation_results["real_cutoff"] - cross_validation_results["cutoff"]
cross_validation_results_s3["sanity check diff"] = cross_validation_results_s3['ds'] - cross_validation_results_s3["cutoff"]


# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(cross_validation_results.sort_values(by='sanity check diff', ascending = False, inplace = False))
# cross_validation_results.sort_values(by='sanity check diff1', ascending = False, inplace = False).head(12)
cross_validation_results_s3.head(2)
cross_validation_results_s3.tail(2)
(cross_validation_results_s3['sanity check diff']/np.timedelta64(1,'Y')).unique()

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(cross_validation_results_s3)

In [ ]:
## calculating mape manually for 10 yr horizon
# _10yfc= cross_validation_results[cross_validation_results['sanity check diff']==timedelta(days=3650)]
# _10yfc['ape'] = abs(100.*(_10yfc['yhat']-_10yfc['y'])/_10yfc['y'])
# _10yfc
# _10yfc['ape'].mean()

In [ ]:
performance_metrics_results_s3 = performance_metrics(cross_validation_results_s3.drop(['real_cutoff', 'sanity check diff'], axis=1, inplace = False))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(performance_metrics_results_s3)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(performance_metrics_results_s3[['mape']].style.hide_index())

In [ ]:
from prophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(cross_validation_results_s3, metric='mape')
# fig = plot_cross_validation_metric(cross_validation_results_s1, metric='rmse')

The MAPE score has range from 0 to infinity, and the lower its value the more accurate the predictions are. <br>
MAPE = (1 / number of forecasted values) x ∑[( |actual - forecast| ) / |actual| ] x 100 <br>
It represents the average of the absolute percentage errors of each entry in a dataset to calculate how accurate the forecasted quantities were in comparison with the actual quantities

The RMSE score has range from 0 to  infinity, and the lower its value the more accurate the predictions are. <br>
Found by squaring the residuals, finding the average of the residuals, and taking the square root of the result:  sqrt(avg((forecastedval - observedval)^2)) from https://www.statisticshowto.com/probability-and-statistics/regression-analysis/rmse-root-mean-square-error/


